In [15]:
import pickle
import tensorflow as tf
import keras
from keras import layers
import scipy.ndimage as ndi
import random
import numpy as np

In [2]:
with open('data.pkl', 'rb') as file:
    dataset_dict = pickle.load(file)

x_train = dataset_dict['x_train']
#x_train_filtered = np.asarray(list(filter(lambda item: item is not None, x_train))).astype('float32')

y_train = dataset_dict['y_train']

x_val = dataset_dict['x_val']
#x_val_filtered = np.asarray(list(filter(lambda item: item is not None, x_val))).astype('float32')

y_val = dataset_dict['y_val']

In [5]:
@tf.function
def rotate(volume):
    """Rotate the volume by a few degrees"""

    def scipy_rotate(volume):
        # define some rotation angles
        angles = [-20, -10, -5, 5, 10, 20]
        # pick angles at random
        angle = random.choice(angles)
        # rotate volume
        volume = ndi.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume

def train_preprocessing(volume, label):
    """Process training data by rotating and adding a channel."""
    # Rotate volume
    volume = rotate(volume)
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

In [7]:
# Define data loaders.
train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((x_val, y_val))

batch_size = 2
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(x_train))
    .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(x_val))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

def get_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model

In [16]:
# Build model.
model = get_model(width=128, height=128, depth=64)
model.summary()

# Compile model.
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

# Train the model, doing validation at the end of each epoch
epochs = 100
model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    shuffle=True,
    verbose=2,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128, 128, 64, 1   0         
                             )]                                  
                                                                 
 conv3d_8 (Conv3D)           (None, 126, 126, 62, 64   1792      
                             )                                   
                                                                 
 max_pooling3d_8 (MaxPoolin  (None, 63, 63, 31, 64)    0         
 g3D)                                                            
                                                                 
 batch_normalization_8 (Bat  (None, 63, 63, 31, 64)    256       
 chNormalization)                                                
                                                                 
 conv3d_9 (Conv3D)           (None, 61, 61, 29, 64)    110656

/home/lodhar/.local/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


144/144 - 57s - loss: 0.7124 - acc: 0.5069 - val_loss: 0.7724 - val_acc: 0.1792 - 57s/epoch - 396ms/step
Epoch 3/10
144/144 - 58s - loss: 0.6979 - acc: 0.5104 - val_loss: 0.8615 - val_acc: 0.2799 - 58s/epoch - 400ms/step
Epoch 4/10
144/144 - 58s - loss: 0.6941 - acc: 0.5451 - val_loss: 0.7394 - val_acc: 0.4182 - 58s/epoch - 403ms/step
Epoch 5/10
144/144 - 58s - loss: 0.7037 - acc: 0.4931 - val_loss: 0.6319 - val_acc: 0.7390 - 58s/epoch - 400ms/step
Epoch 6/10
144/144 - 59s - loss: 0.6882 - acc: 0.5521 - val_loss: 0.7324 - val_acc: 0.5220 - 59s/epoch - 408ms/step
Epoch 7/10
144/144 - 56s - loss: 0.6824 - acc: 0.5799 - val_loss: 0.8476 - val_acc: 0.3711 - 56s/epoch - 388ms/step
Epoch 8/10
144/144 - 56s - loss: 0.6837 - acc: 0.5556 - val_loss: 0.7621 - val_acc: 0.4748 - 56s/epoch - 386ms/step
Epoch 9/10
144/144 - 53s - loss: 0.6795 - acc: 0.5521 - val_loss: 1.4458 - val_acc: 0.2233 - 53s/epoch - 371ms/step
Epoch 10/10
144/144 - 52s - loss: 0.7011 - acc: 0.5347 - val_loss: 1.4813 - val_acc